## Comparing different groups of experiments


This notebook compares different groups of experiments, using the metrics described in each cell.

Each group takes a dictionary of `'experiment name': list[Experiment]`.
First, create each group by changing the experiment names in the lists on the next cell (lines 10 and 11), and then add each Group object to the `groups` list (last line of next cell).

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from pasna_analysis import DataLoader, Experiment, ExperimentConfig, Group


wt_config  = {
    '20240514': ExperimentConfig(first_peak_threshold=30, to_exclude=[2, 4])
}

mut_config = {
    '20240611': ExperimentConfig(first_peak_threshold=30, to_exclude=[])
}

wt_experiments = {}
for exp, config in wt_config.items():
    exp_path = Path.cwd().parent.joinpath('data', exp)
    wt_experiments[exp] = Experiment(DataLoader(exp_path), config.first_peak_threshold, config.to_exclude, config.has_transients)

mut_experiments = {}
for exp, config in mut_config.items():
    exp_path = Path.cwd().parent.joinpath('data', exp)
    mut_experiments[exp] = Experiment(DataLoader(exp_path), config.first_peak_threshold, config.to_exclude)

wt = Group('wt', wt_experiments)
mut = Group('mutant', mut_experiments)

groups = [wt, mut]

In [ ]:
'''Developmental times at first peak.'''
data = {'dev_fp': [], 'group': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            time_first_peak = trace.peak_times[0]
            dev_time_first_peak = emb.get_DT_from_time(time_first_peak)

            data['dev_fp'].append(dev_time_first_peak)
            data['group'].append(group.name)

df = pd.DataFrame(data)
group_avgs = {'group': [], 'fp_avg': []}

for group in groups:
    g = df[df['group']==group.name]
    group_avgs['group'].append(group.name)
    group_avgs['fp_avg'].append(g['dev_fp'].mean())

sns.set_theme(style='darkgrid')
# sns.catplot(group_avgs, x='group', y='fp_avg', kind='bar', width=0.2, linewidth=1, color='0.7')
ax = sns.swarmplot(data=data, x='group', y='dev_fp', hue='group')
ax.set_title('Developmental time at first peak')
ax.set_ylabel('Developmental time')
ax.set_xlabel('Experiment Group');

In [ ]:
'''CDF of peak developmental times'''
data = {'dev_time': [], 'group': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            dev_times = [emb.get_DT_from_time(t) for t in trace.peak_times]
            data['dev_time'].extend(dev_times)
            data['group'].extend([group.name]*len(dev_times))

sns.set_theme(style='darkgrid')
ax = sns.ecdfplot(data=data, x='dev_time', hue='group')
ax.set_xlim([1.9, 2.9])
ax.set_title('CDF developmental times of peaks')
ax.set_ylabel('Proportion')
ax.set_xlabel('Developmental Time');

In [ ]:
'''Peak amplitudes for each episode.'''
num_of_peaks = 13
data = {'peak_amp': [], 'group': [], 'peak_idx': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            t = exp.traces[emb.name]
            for i, amp in zip(range(num_of_peaks), t.peak_amplitudes):
                data['peak_amp'].append(amp)
                data['group'].append(group.name)
                data['peak_idx'].append(i)

amps = pd.DataFrame(data)

sns.set_theme(style='darkgrid')
ax = sns.lineplot(data=data, x='peak_idx', y='peak_amp', hue='group')
ax.set_title(f'Peak amplitudes')
ax.set_xlabel('Peak number')
ax.set_ylabel('\u0394F/F');

In [ ]:
'''Developmental time for each episode.'''
data = {'group': [], 'dev_time': [], 'idx': []}

for group in groups:
    for exp in group.experiments.values():
        for emb_n, emb in enumerate(exp.embryos):
            trace = exp.traces[emb.name]
            for i, t in zip(range(12), trace.peak_times):
                data['group'].append(group.name)
                data['dev_time'].append(emb.get_DT_from_time(t))
                data['idx'].append(i)

dev_times = pd.DataFrame(data)

f, ax = plt.subplots(figsize=(10,6))
sns.set_theme(style='darkgrid')
ax = sns.pointplot(data=dev_times, x='idx', y='dev_time', hue='group', alpha=0.7, errorbar='pi')
ax.set_title('Developmental time for each peak')
ax.set_xlabel('Peak #')
ax.set_ylabel('Developmental time');

In [ ]:
'''Intervals between each episode.'''
data = {'group': [], 'interval': [], 'idx': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, interval in zip(range(12), trace.peak_intervals):
                data['group'].append(group.name)
                data['interval'].append(interval/60)
                data['idx'].append(i+1)

amps = pd.DataFrame(data)

f, ax = plt.subplots(figsize=(10,6))
sns.set_theme(style='darkgrid')
ax = sns.pointplot(data=amps, x='idx', y='interval', hue='group')
ax.set_title('Intervals by peak')
ax.set_xlabel('Interval #')
ax.set_ylabel('Interval (min)');

In [ ]:
'''Duration of each peak.'''
data = {'group': [], 'duration': [], 'idx': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, duration in zip(range(12), trace.peak_durations):
                data['group'].append(group.name)
                data['duration'].append(duration/60)
                data['idx'].append(i+1)

amps = pd.DataFrame(data)

f, ax = plt.subplots(figsize=(10,6))
sns.set_theme(style='darkgrid')
ax = sns.pointplot(data=amps, x='idx', y='duration', hue='group')
ax.set_title('Durations by peak')
ax.set_xlabel('Peak #')
ax.set_ylabel('Duration (min)');

In [ ]:
'''Rise times'''
data = {'group': [], 'rise_times': [], 'idx': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, rise in zip(range(12), trace.peak_rise_times):
                data['group'].append(group.name)
                data['rise_times'].append(rise)
                data['idx'].append(i+1)

amps = pd.DataFrame(data)

f, ax = plt.subplots(figsize=(10,6))
sns.set_theme(style='darkgrid')
ax = sns.pointplot(data=amps, x='idx', y='rise_times', hue='group')
ax.set_title('Peak rise times')
ax.set_xlabel('Peak #')
ax.set_ylabel('Duration (min)');

In [ ]:
'''Decay times'''
data = {'group': [], 'decay_times': [], 'idx': []}

for group in groups:
    for exp in group.experiments.values():
        for emb in exp.embryos:
            trace = exp.traces[emb.name]
            for i, decay in zip(range(12), trace.peak_decay_times):
                data['group'].append(group.name)
                data['decay_times'].append(decay)
                data['idx'].append(i+1)

amps = pd.DataFrame(data)

f, ax = plt.subplots(figsize=(10,6))
sns.set_theme(style='darkgrid')
ax = sns.pointplot(data=amps, x='idx', y='decay_times', hue='group')
ax.set_title('Peak decay times')
ax.set_xlabel('Peak #')
ax.set_ylabel('Duration (min)');